In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
pd.set_option('display.max_colwidth', 1500)

In [3]:
df = pd.read_csv('../axolotl.test.ru.tsv', sep='\t')
print(df.date.value_counts())
df = df[df['example'].notna()]
print(df.date.value_counts())
df.reset_index(inplace=True)

date
new    1702
old     424
Name: count, dtype: int64
date
new    1702
old     288
Name: count, dtype: int64


In [4]:
df = pd.read_csv('../axolotl.test.ru.tsv', sep='\t')
print(df.date.value_counts())
df = df[df['example'].notna()]
print(df.date.value_counts())
df.reset_index(inplace=True)

with open('ru_train_orths.txt', 'w') as f:
    for orth in df['orth'].unique():
        f.write(f'{orth.split()[0]}\n')
with open('ru_train_words.txt', 'w') as f:
    for word in df['word'].unique():
        f.write(f'{word}\n')
with open('ru_train_examples.txt', 'w') as f:
    for example in df['example']:
        f.write(f'{example}\n')

!python word_form_generator.py -l ru_train_words.txt -c ru_train_examples.txt -i index_ru_train_words.pkl
!mv forms.txt ru_train_word_forms.txt
!python word_form_generator.py -l ru_train_orths.txt -c ru_train_examples.txt -i index_ru_train_orths.pkl
!mv forms.txt ru_train_orth_forms.txt
!python word_uses_retriever.py -c ru_train_examples.txt -f ru_train_word_forms.txt -i index_ru_train_words.pkl
!mv uses uses_ru_train_word
!python word_uses_retriever.py -c ru_train_examples.txt -f ru_train_orth_forms.txt -i index_ru_train_orths.pkl
!mv uses uses_ru_train_orth

df1 = df.copy()
df2 = df.copy()

for f in os.listdir('uses_ru_train_word'):
    usages = pd.read_csv('uses_ru_train_word/' + f, sep='\t')
    for word, context_id, positions in zip(usages.word, usages.context_id, usages.positions):
        if df1['word'][int(context_id.split('_')[0])] == word:
            df1.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions

for f in os.listdir('uses_ru_train_orth'):
    usages = pd.read_csv('uses_ru_train_orth/' + f, sep='\t')
    for orth, context_id, positions in zip(usages.word, usages.context_id, usages.positions):
        if df2['orth'][int(context_id.split('_')[0])].split()[0] == orth:
            df2.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions

date
new    1702
old     424
Name: count, dtype: int64
date
new    1702
old     288
Name: count, dtype: int64
Traceback (most recent call last):
  File "/home/jovyan/kokosinskiy/projects/axolotl24_shared_task/data/russian/add_index/word_form_generator.py", line 5, in <module>
    import rutokenizer
ModuleNotFoundError: No module named 'rutokenizer'
mv: cannot stat 'forms.txt': No such file or directory
Traceback (most recent call last):
  File "/home/jovyan/kokosinskiy/projects/axolotl24_shared_task/data/russian/add_index/word_form_generator.py", line 5, in <module>
    import rutokenizer
ModuleNotFoundError: No module named 'rutokenizer'
mv: cannot stat 'forms.txt': No such file or directory
Traceback (most recent call last):
  File "/home/jovyan/kokosinskiy/projects/axolotl24_shared_task/data/russian/add_index/word_uses_retriever.py", line 63, in <module>
    main()
  File "/home/jovyan/kokosinskiy/projects/axolotl24_shared_task/data/russian/add_index/word_uses_retriever.py", line 52

FileNotFoundError: [Errno 2] No such file or directory: 'uses_ru_train_word'

In [15]:
df = pd.read_csv('ru_train_with_indices.tsv', sep='\t')
df[df.indices_target_token.isna()]
df

,Unnamed: 0,index,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,0,1,train_ru_1,миро,миро,miro_8igZuzZK97Q,"религ. жидкое ароматическое масло, освящаемое патриархом или митрополитом, используемое в таинстве миропомазания, при освящении престолов и антиминсов православных храмов, при венчании монарха на царство","Затем ключарь приглашает, чтобы женщины вышли из алтаря… Подаёт святое миро. Архиерей помазует крестообразно сначала трапезу в тех местах, где во время литургии стоит евангелие, дискос и потир.",71-74,new,new
1,1,2,train_ru_2,миро,миро,miro_8igZuzZK97Q,"религ. жидкое ароматическое масло, освящаемое патриархом или митрополитом, используемое в таинстве миропомазания, при освящении престолов и антиминсов православных храмов, при венчании монарха на царство","Они везде одним мvром мазаны. Мvром покрыт (т. е. помазан), с миром заспит. Рогожцы в",62-65,old,old
2,2,3,train_ru_3,миро,миро,miro_o98UfpSoYH4,"религ. жидкость, иногда чудесным образом выделяющаяся на иконах и мощах святых","Мощи святого хранятся здесь до сих пор, продолжая источать миро.",59-62,new,new
3,3,4,train_ru_4,миро,миро,miro_o98UfpSoYH4,"религ. жидкость, иногда чудесным образом выделяющаяся на иконах и мощах святых","Иосиф увидел, что по иконе Богородицы текли струйки мира, которое и издавало благоухание.",52-56,new,new
4,4,5,train_ru_14,могильный,могильный,mogil'nyj_GUn5GrgpzqE,"такой, как в могиле, напоминающий чем-либо могилу; полный, абсолютный (о тишине, покое, темноте)","Казалось, перед революцией удесятерилось ожидание, предчувствие, предвидение золотого века, рая на земле; казалось, вся душа нации исступленно бредила и грезила чем-то единым, в котором сплетались нерасторжимо смерть и бессмертие, золотой век и гибель, могильный мрак и ослепительный свет…",253-262,new,new
...,...,...,...,...,...,...,...,...,...,...,...
6088,6088,6489,train_ru_7987,ошибочный,ошибочный,oshibochnyj_cQbSjTxuEU4,"содержащий ошибку, неверный, неправильный","Белинский спорил только против мнений, положительно вредных и решительно ошибочных: нельзя указать ни одного случая, когда бы он считал нужным восставать против убеждений, которые были безвредны или не нелепы.",73-81,new,new
6089,6089,6490,train_ru_7988,ошибочный,ошибочный,oshibochnyj_cQbSjTxuEU4,"содержащий ошибку, неверный, неправильный","Часто думают, что для стариков характерна потеря волос, но мнение это ошибочно, потому что лысеть начинают в более раннем возрасте.",70-78,new,new
6090,6090,6491,train_ru_7989,ошибочный,ошибочный,oshibochnyj_cQbSjTxuEU4,"содержащий ошибку, неверный, неправильный","Рибосома синтезирует белок со средней скоростью 10—20 аминокислот в секунду. Точность трансляции исключительно высока — ошибочное включение «неправильного» аминокислотного остатка в цепь белка составляет в среднем одну аминокислоту на 3 тыс. звеньев (при средней длине белковой цепи у человека в 500 аминокислотных остатков), т. е. всего одна ошибка на шесть белков.",120-129,new,new
6091,6091,6492,train_ru_7990,ошибочный,ошибочный,oshibochnyj_cQbSjTxuEU4,"содержащий ошибку, неверный, неправильный",Ошибочное,0-9,old,old


In [26]:
df = pd.read_csv('../axolotl.dev.ru.tsv', sep='\t')
print(df.date.value_counts())
df = df[df['example'].notna()]
print(df.date.value_counts())
df.reset_index(inplace=True)

with open('ru_dev_orths.txt', 'w') as f:
    for orth in df['orth'].unique():
        f.write(f'{orth.split()[0]}\n')
with open('ru_dev_words.txt', 'w') as f:
    for word in df['word'].unique():
        f.write(f'{word}\n')
with open('ru_dev_examples.txt', 'w') as f:
    for example in df['example']:
        f.write(f'{example}\n')

!python word_form_generator.py -l ru_dev_words.txt -c ru_dev_examples.txt -i index_ru_dev_words.pkl
!mv forms.txt ru_dev_word_forms.txt
!python word_form_generator.py -l ru_dev_orths.txt -c ru_dev_examples.txt -i index_ru_dev_orths.pkl
!mv forms.txt ru_dev_orth_forms.txt
!python word_uses_retriever.py -c ru_dev_examples.txt -f ru_dev_word_forms.txt -i index_ru_dev_words.pkl
!mv uses uses_ru_dev_word
!python word_uses_retriever.py -c ru_dev_examples.txt -f ru_dev_orth_forms.txt -i index_ru_dev_orths.pkl
!mv uses uses_ru_dev_orth

df1 = df.copy()
df2 = df.copy()

for f in os.listdir('uses_ru_dev_word'):
    usages = pd.read_csv('uses_ru_dev_word/' + f, sep='\t')
    for word, context_id, positions in zip(usages.word, usages.context_id, usages.positions):
        if df1['word'][int(context_id.split('_')[0])] == word:
            df1.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions

for f in os.listdir('uses_ru_dev_orth'):
    usages = pd.read_csv('uses_ru_dev_orth/' + f, sep='\t')
    for orth, context_id, positions in zip(usages.word, usages.context_id, usages.positions):
        if df2['orth'][int(context_id.split('_')[0])].split()[0] == orth:
            df2.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions

date
new    1605
old     421
Name: count, dtype: int64
date
new    1605
old     307
Name: count, dtype: int64
Starting word form generation..
100%|███████████████████████████████████████| 1912/1912 [00:38<00:00, 49.67it/s]
Word form generation fnished!
Starting word form generation..
100%|███████████████████████████████████████| 1912/1912 [01:09<00:00, 27.52it/s]
Word form generation fnished!
1912it [00:00, 1319215.21it/s]
100%|█████████████████████████████████████████| 201/201 [00:21<00:00,  9.48it/s]
Uses retrieval finished!
1912it [00:00, 1287860.81it/s]
100%|█████████████████████████████████████████| 201/201 [00:18<00:00, 10.93it/s]
Uses retrieval finished!


/tmp/ipykernel_6887/673676271.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '25-35' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df1.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions
/tmp/ipykernel_6887/673676271.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '25-35' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df2.loc[int(context_id.split('_')[0]), 'indices_target_token'] = positions


In [27]:
df1.fillna(df2)

,index,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,0,dev_ru_0,могильник,могильникъ,mogil'nik_UYTE5-B076I,"вят. слово из Вятской губернии арх. слово из Архангельской губернии кур.? могила, в знач. значение насыпи над покойником, дерновая кладка, памятник","могилки, кладбище",NaN,old,old
1,3,dev_ru_3,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,"Орёл изображает реку Халзан, его голова ― скалу на Халзане, а лапы могильника ― могилу на скале.",67-76,new,new
2,4,dev_ru_4,могильник,могильникъ,mogil'nik_ahboIs9hMMk,орнитол. хищная птица рода крупных степных орлов,Орел могильник,5-14,old,old
3,5,dev_ru_5,могильник,могильникъ,mogil'nik_2g5paR8fF5I,"могильщик, могиляк, могилокопатель гробокоп, рабочий при кладбище.","Это я, я, который в """"Гамлете"""" могильщика играл!""""",NaN,old,old
4,7,dev_ru_7,могильник,могильникъ,mogil'nik_FciH4mIkAEU,"археол. в древности — комплекс многих погребений, захоронений",В экспозиции представлен инвентарь из реликтового языческого балтского женского захоронения в курганном могильнике конца XIII ― начала XIV века.,104-113,new,new
...,...,...,...,...,...,...,...,...,...,...
1907,2021,dev_ru_2456,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение; одержимость",Надо было опять торопиться. Но недавняя горячка прошла.,40-47,new,new
1908,2022,dev_ru_2457,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение; одержимость","И вторая нога вошла в чулок, вернее, чулок пополз по ней, а взмокшие ладони дяди Були, забыв горячку страха, обрели горячку события.",116-123,new,new
1909,2023,dev_ru_2458,горячка,горячка,gorjachka_8bgKSQYNDr4,"перен. разг. страстное увлечение, возбуждение; одержимость","В порыве своей первой творческой горячки он вообразил, что, уединившись куда-нибудь на один год с картиной, он в этот срок совершенно закончит её…",33-40,new,new
1910,2024,dev_ru_2459,горячка,горячка,gorjachka_re-9L7XQcEo,"перен. напряженная работа, требующая немедленного исполнения; спешка",В суматохе и горячке последних часов кого-нибудь найти или что-нибудь узнать практически невозможно,13-20,new,new


In [28]:
df1.fillna(df2)[df1.fillna(df2).indices_target_token.isna()].period.value_counts()

period
old    61
new     3
Name: count, dtype: int64

In [16]:
df.period.value_counts()

period
new    4581
old    1512
Name: count, dtype: int64

In [14]:
61/1600

0.038125

In [17]:
df[df.indices_target_token.isna()]['period'].value_counts()

period
old    378
new     17
Name: count, dtype: int64

In [30]:
df1.fillna(df2).to_csv('ru_dev_with_indices.tsv', sep='\t')

In [9]:
df = df1.fillna(df2)

In [10]:
df = df[df['indices_target_token'].notna()]

In [16]:
df.to_csv('ru_train_wsi.tsv', sep='\t')

In [13]:
df['context_id'] = df['index']
df['gold_sense_id'] = df['sense_id']
df['predict_sense_id'] =  None
df['positions'] = df['indices_target_token']
df['context'] = df['example']

In [14]:
df = df[['context_id', 'word', 'gold_sense_id', 'predict_sense_id', 'positions', 'context']]

In [15]:
df.to_csv(

,context_id,word,gold_sense_id,predict_sense_id,positions,context
0,1,миро,miro_8igZuzZK97Q,None,72-76,"Затем ключарь приглашает , чтобы женщины вышли из алтаря… Подаёт святое миро . Архиерей помазует крестообразно сначала трапезу в тех местах , где во время литургии стоит евангелие , дискос и потир ."
2,3,миро,miro_o98UfpSoYH4,None,60-64,"Мощи святого хранятся здесь до сих пор , продолжая источать миро ."
3,4,миро,miro_o98UfpSoYH4,None,53-57,"Иосиф увидел , что по иконе Богородицы текли струйки мира , которое и издавало благоухание ."
4,5,могила,mogila_WTu4h3aBxzE,None,28-34,"Сын моря , средь морей твоя могила ! // Вот мщение за муки стольких дней !"
6,9,могила,mogila_bBHZ-GD99Pc,None,143-149,"Неизвестно , что бы он сделал , как вдруг мост заколебался у него под ногами ; он оглянулся : боже мой ! он на краю пропасти : перед ним зияет могила : половина моста отделилась и отплывает прочь…"
...,...,...,...,...,...,...
7493,7986,ошибочный,oshibochnyj_cQbSjTxuEU4,None,86-94,Для алгоритма обучения персептрона важно иметь быструю сходимость и малые вероятности ошибочных решений на этапе распознавания .
7494,7987,ошибочный,oshibochnyj_cQbSjTxuEU4,None,74-82,"Белинский спорил только против мнений , положительно вредных и решительно ошибочных : нельзя указать ни одного случая , когда бы он считал нужным восставать против убеждений , которые были безвредны или не нелепы ."
7495,7988,ошибочный,oshibochnyj_cQbSjTxuEU4,None,72-80,"Часто думают , что для стариков характерна потеря волос , но мнение это ошибочно , потому что лысеть начинают в более раннем возрасте ."
7496,7989,ошибочный,oshibochnyj_cQbSjTxuEU4,None,121-130,"Рибосома синтезирует белок со средней скоростью 10—20 аминокислот в секунду . Точность трансляции исключительно высока — ошибочное включение `` неправильного '' аминокислотного остатка в цепь белка составляет в среднем одну аминокислоту на 3 тыс . звеньев ( при средней длине белковой цепи у человека в 500 аминокислотных остатков ) , т. е. всего одна ошибка на шесть белков ."


In [6]:
df = pd.read_csv('../../finnish/axolotl.train.fi.tsv', sep='\t')

In [7]:
df

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period
0,train_fi_0,mies-luku,"mieslucuns,",mies-luku_tguFpsBi4m0,"suku-, väestöluettelo","minä löysin heidän mieslucuns, cuin ennen olit tullet fangeudest",19:30,1600,old
1,train_fi_1,mies-luku,Mies lucu,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku",ylisti Theudas itzens – – Ja henen tygens langesi Mies lucu lehes nelie sata,50:59,1543,old
2,train_fi_2,mies-luku,mies,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku",[Sakot] pitä mies lughun iälkin vlos maxettaman,13:17,1543,old
3,train_fi_3,mies-luku,mies,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku","tule kihlacunnan nimitöxen mies lughun nimittä, ia iocaitzen tule murha sakonn maxa",27:31,1543,old
4,train_fi_4,mies-luku,mies lucu,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku","mies lucu tuli wijsi tuhatta, iotca sanan cwlit ia uscoit",0:9,1600,old
...,...,...,...,...,...,...,...,...,...
93134,train_fi_93134,mies,myös,mies_-Lg9dsyT9IU,"itsenäinen, taloudellisesti riippumaton ihminen, oma herransa; )( toisen alainen, palvelija","tulewat myös caicki muut työntekiät – – jotca Manufacturein ja Fabriquein alla owat, tästä Ulosteosta – – wapaaxi, jos ei he ole itze miehensä, eli tee työtä oman kätensä päälle",8:12,1700,new
93135,train_fi_93135,mies,se,mies_-Lg9dsyT9IU,"itsenäinen, taloudellisesti riippumaton ihminen, oma herransa; )( toisen alainen, palvelija","jos se on yxi Förlags Mestari – – joca estää hänen Edelläkätten-maxajansa cajomasta itze walmijxi tekemiseen, nijn se – – [ei] ole ikänänsä mahdollinen pääsemään omaxi miehexensä",4:6,1750,new
93136,train_fi_93136,mies,miehen,mies_lTWCX9DhlYs,sivullinen; lak.joka ei (alun perin) ole osapuolena asiassa,"Duomitan calu colmannen miehen taca, se pitä waarisa pidettämän",24:30,1700,new
93137,train_fi_93137,mies,mies:,mies_lTWCX9DhlYs,sivullinen; lak.joka ei (alun perin) ole osapuolena asiassa,"Colmas mies: jos asia händä – – coske, cuinga hän mahta, rijta-cumpanein rijdan alla, itzens edesandaa",7:12,1750,new


In [8]:
df['context_id'] = df.index
df['gold_sense_id'] = df['sense_id']
df['predict_sense_id'] =  None
df['positions'] = list(map(lambda x: '-'.join(x.split(':')), df['indices_target_token']))
df['context'] = df['example']

In [9]:
df['gold_sense_id'] = df['gold_sense_id'].apply(lambda x: ''.join(x.split()))

In [10]:
df['gold_sense_id'][df['gold_sense_id'].apply(lambda x:  ' ' in x)]

Series([], Name: gold_sense_id, dtype: object)

In [11]:
np.any(df['gold_sense_id'].apply(lambda x: x is not None and ' ' in x))

False

In [19]:
df1 = df.copy()
df1.loc[df1.period == 'old', 'gold_sense_id'] = None
df1[df1.period == 'old'][['context_id', 'word', 'gold_sense_id', 'predict_sense_id', 'positions', 'context']].to_csv('fi_train_wsi_old_v2.tsv', sep='\t')
df1[df1.period == 'new'][['context_id', 'word', 'gold_sense_id', 'predict_sense_id', 'positions', 'context']].to_csv('fi_train_wsi_new_v2.tsv', sep='\t')

In [16]:
df.to_csv("./../../../../summer-wsi/datasets/axolotl24/fi/fi_train_mixed.tsv", sep='\t', index=False)

In [12]:
df

,usage_id,word,orth,sense_id,gloss,example,indices_target_token,date,period,context_id,gold_sense_id,predict_sense_id,positions,context
0,train_fi_0,mies-luku,"mieslucuns,",mies-luku_tguFpsBi4m0,"suku-, väestöluettelo","minä löysin heidän mieslucuns, cuin ennen olit tullet fangeudest",19:30,1600,old,0,mies-luku_tguFpsBi4m0,None,19-30,"minä löysin heidän mieslucuns, cuin ennen olit tullet fangeudest"
1,train_fi_1,mies-luku,Mies lucu,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku",ylisti Theudas itzens – – Ja henen tygens langesi Mies lucu lehes nelie sata,50:59,1543,old,1,mies-luku_XeagDiyRXpA,None,50-59,ylisti Theudas itzens – – Ja henen tygens langesi Mies lucu lehes nelie sata
2,train_fi_2,mies-luku,mies,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku",[Sakot] pitä mies lughun iälkin vlos maxettaman,13:17,1543,old,2,mies-luku_XeagDiyRXpA,None,13-17,[Sakot] pitä mies lughun iälkin vlos maxettaman
3,train_fi_3,mies-luku,mies,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku","tule kihlacunnan nimitöxen mies lughun nimittä, ia iocaitzen tule murha sakonn maxa",27:31,1543,old,3,mies-luku_XeagDiyRXpA,None,27-31,"tule kihlacunnan nimitöxen mies lughun nimittä, ia iocaitzen tule murha sakonn maxa"
4,train_fi_4,mies-luku,mies lucu,mies-luku_XeagDiyRXpA,"miesjoukko; miesten lukumäärä, pääluku","mies lucu tuli wijsi tuhatta, iotca sanan cwlit ia uscoit",0:9,1600,old,4,mies-luku_XeagDiyRXpA,None,0-9,"mies lucu tuli wijsi tuhatta, iotca sanan cwlit ia uscoit"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93134,train_fi_93134,mies,myös,mies_-Lg9dsyT9IU,"itsenäinen, taloudellisesti riippumaton ihminen, oma herransa; )( toisen alainen, palvelija","tulewat myös caicki muut työntekiät – – jotca Manufacturein ja Fabriquein alla owat, tästä Ulosteosta – – wapaaxi, jos ei he ole itze miehensä, eli tee työtä oman kätensä päälle",8:12,1700,new,93134,mies_-Lg9dsyT9IU,None,8-12,"tulewat myös caicki muut työntekiät – – jotca Manufacturein ja Fabriquein alla owat, tästä Ulosteosta – – wapaaxi, jos ei he ole itze miehensä, eli tee työtä oman kätensä päälle"
93135,train_fi_93135,mies,se,mies_-Lg9dsyT9IU,"itsenäinen, taloudellisesti riippumaton ihminen, oma herransa; )( toisen alainen, palvelija","jos se on yxi Förlags Mestari – – joca estää hänen Edelläkätten-maxajansa cajomasta itze walmijxi tekemiseen, nijn se – – [ei] ole ikänänsä mahdollinen pääsemään omaxi miehexensä",4:6,1750,new,93135,mies_-Lg9dsyT9IU,None,4-6,"jos se on yxi Förlags Mestari – – joca estää hänen Edelläkätten-maxajansa cajomasta itze walmijxi tekemiseen, nijn se – – [ei] ole ikänänsä mahdollinen pääsemään omaxi miehexensä"
93136,train_fi_93136,mies,miehen,mies_lTWCX9DhlYs,sivullinen; lak.joka ei (alun perin) ole osapuolena asiassa,"Duomitan calu colmannen miehen taca, se pitä waarisa pidettämän",24:30,1700,new,93136,mies_lTWCX9DhlYs,None,24-30,"Duomitan calu colmannen miehen taca, se pitä waarisa pidettämän"
93137,train_fi_93137,mies,mies:,mies_lTWCX9DhlYs,sivullinen; lak.joka ei (alun perin) ole osapuolena asiassa,"Colmas mies: jos asia händä – – coske, cuinga hän mahta, rijta-cumpanein rijdan alla, itzens edesandaa",7:12,1750,new,93137,mies_lTWCX9DhlYs,None,7-12,"Colmas mies: jos asia händä – – coske, cuinga hän mahta, rijta-cumpanein rijdan alla, itzens edesandaa"
